# 1. K-means로 학습하기 (iris)

In [6]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np

In [10]:
# data load

iris = load_wine()

iris.data = pd.DataFrame(wine.data)
iris.target = pd.DataFrame(wine.target)

In [8]:
# 10번 반복하는 pipeline 함수 제작

from sklearn.pipeline import Pipeline

def pipe(model: Pipeline, train, target):
    result = []
    for i in range(0,10,1):
        model.fit_transform(train, target)
        result.append(model.predict(train))
    return result

In [11]:
# normal_iris_kmeans accuracy

from sklearn.cluster import KMeans

pipeline = Pipeline([('Kmeans', KMeans(n_clusters=3, init='random'))]) # normal predict 생성
result_norm = pipe(pipeline, iris.data, iris.target)


from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score # Adjusted rand score는 음수가 나올수 있으므로 백분율 계산을 위해서 Normalized mutual information을 사용했다. 

score = []
for i in result_norm:
    score.append(normalized_mutual_info_score(iris.target[0], i))

print(np.mean(score))

0.4287568597645355


In [15]:
# PCA _iris_kemans accuracy

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

# 각 정규화 된 값들을 PCA로 차원축소 시키고 값이 어떻게 변화하는지 살펴본다.
# 일단 컴포넌트는 2로 하고 1~3중 어느값이 가장 좋은 결과를 가지는지 학습시켜본다.
pipeline_minmax = Pipeline([('MinMax', MinMaxScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_maxabs = Pipeline([('MaxAbs', MaxAbsScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_standard = Pipeline([('Standard', StandardScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_robust = Pipeline([('Robust', RobustScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_normalize = Pipeline([('Normalize', Normalizer()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_log = Pipeline([('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])

result_minmax = pipe(pipeline_minmax, iris.data, iris.target)
result_maxabs = pipe(pipeline_maxabs, iris.data, iris.target)
result_standard = pipe(pipeline_standard, iris.data, iris.target)
result_robust = pipe(pipeline_robust, iris.data, iris.target)
result_normalize = pipe(pipeline_normalize, iris.data, iris.target)
result_log = pipe(pipeline_log, np.log1p(iris.data), iris.target)

score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []
for i in range(0,10,1):
    score_minmax.append(normalized_mutual_info_score(iris.target[0], result_minmax[i]))
    score_maxabs.append(normalized_mutual_info_score(iris.target[0], result_maxabs[i]))
    score_standard.append(normalized_mutual_info_score(iris.target[0], result_standard[i]))
    score_robust.append(normalized_mutual_info_score(iris.target[0], result_robust[i]))
    score_normalize.append(normalized_mutual_info_score(iris.target[0], result_normalize[i]))
    score_log.append(normalized_mutual_info_score(iris.target[0], result_log[i]))

print('minmax_pca:',np.mean(score_minmax))
print('maxabs_pca:',np.mean(score_maxabs))
print('standard_pca:',np.mean(score_standard))
print('robust_pca:',np.mean(score_robust))
print('normalize_pca:',np.mean(score_normalize))
print('log_scaled_pca:',np.mean(score_log))

# Standard의 accuracy가 가장 높으므로 Standard로 전처리된 값을 이용해 PCA의 컴포넌트를 1~3까지 넣어본다.

minmax_pca: 0.8346588494891206
maxabs_pca: 0.7659600030261117
standard_pca: 0.8797202687466328
robust_pca: 0.7429987853585629
normalize_pca: 0.382280412585774
log_scaled_pca: 0.8388362445743244


In [16]:
# PCA _iris_kemans accuracy (change_component)

pipeline1 = Pipeline([('Standard', StandardScaler()),('PCA', PCA(n_components=1)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline2 = Pipeline([('Standard', StandardScaler()),('PCA', PCA(n_components=2)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline3 = Pipeline([('Standard', StandardScaler()),('PCA', PCA(n_components=3)),('Kmeans', KMeans(n_clusters=3, init='random'))])

result1 = pipe(pipeline1, iris.data, iris.target)
result2 = pipe(pipeline2, iris.data, iris.target)
result3 = pipe(pipeline3, iris.data, iris.target)

score1, score2, score3 = [], [], []

for i in range(0,10,1):
    score1.append(normalized_mutual_info_score(iris.target[0], result1[i]))
    score2.append(normalized_mutual_info_score(iris.target[0], result2[i]))
    score3.append(normalized_mutual_info_score(iris.target[0], result3[i]))

print('component=1:',np.mean(score1))
print('component=2:',np.mean(score2))
print('component=3:',np.mean(score3))

# component는 1개일 때 가장 좋은 결과를 보였다. 다만 dataset 마다 결과는 다를 것으로보인다.

component=1: 0.528666524215945
component=2: 0.8776082230529975
component=3: 0.8684196027882594


In [17]:
# TSNE는 predict가 없기 때문에 pipeline에 넣을 경우 오류가 생긴다. 따라서 이를 우회해줄 TSNE Handler 함수를 만든다. (https://data-newbie.tistory.com/568)

from sklearn.manifold import TSNE
from sklearn.base import BaseEstimator, TransformerMixin

class TSNEHandler(BaseEstimator, TransformerMixin):
    def __init__(self,n_components=2, random_state=0): # randomstate는 0으로 고정해서 결과값을 통일한다.
        self.tsne =TSNE(n_components=n_components, random_state = random_state, method="exact")
    
    def fit(self,x):
        self.tsne.fit(x)
        
    def transform(self,x):
        x = self.tsne.fit_transform(x)
        return x
        
    def fit_transform(self,x,y):
        x = self.tsne.fit_transform(x)    
        return x

In [18]:
# TSNE_iris_kmeans accuracy
# TSNE의 경우 코드 실행에 시간이 오래 걸리므로 한번의 학습만 실행

pipeline_minmax = Pipeline([('MinMax', MinMaxScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_maxabs = Pipeline([('MaxAbs', MaxAbsScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_standard = Pipeline([('Standard', StandardScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_robust = Pipeline([('Robust', RobustScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_normalize = Pipeline([('Normalize', Normalizer()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline_log = Pipeline([('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])

result_minmax = pipeline_minmax.fit_predict(iris.data, iris.target)
result_maxabs = pipeline_maxabs.fit_predict(iris.data, iris.target)
result_standard = pipeline_standard.fit_predict(iris.data, iris.target)
result_robust = pipeline_robust.fit_predict(iris.data, iris.target)
result_normalize = pipeline_normalize.fit_predict(iris.data, iris.target)
result_log = pipeline_log.fit_predict(iris.data, iris.target)

score_minmax = normalized_mutual_info_score(iris.target[0], result_minmax)
score_maxabs = normalized_mutual_info_score(iris.target[0], result_maxabs)
score_standard = normalized_mutual_info_score(iris.target[0], result_standard)
score_robust = normalized_mutual_info_score(iris.target[0], result_robust)
score_normalize = normalized_mutual_info_score(iris.target[0], result_normalize)
score_log = normalized_mutual_info_score(iris.target[0], result_log)

print('minmax_TSNE:',score_minmax)
print('maxabs_TSNE:',score_maxabs)
print('standard_TSNE:',score_standard)
print('robust_TSNE:',score_robust)
print('normalize_TSNE:',score_normalize)
print('log_scaled_TSNE:',score_log)

# 이번에는 minmax의 accuracy가 가장 높다. minmax로 전처리된 값을 이용해 TSNE의 컴포넌트를 1~3까지 넣어본다.

minmax_TSNE: 0.8251627219307877
maxabs_TSNE: 0.7412413931124873
standard_TSNE: 0.784395474761121
robust_TSNE: 0.7727542291141806
normalize_TSNE: 0.37098873026498574
log_scaled_TSNE: 0.38621844619687873


In [19]:
# TSNE_iris_kmeans accuracy (change_component)

pipeline1 = Pipeline([('MinMax', MinMaxScaler()),('t-SNE', TSNEHandler(n_components=1, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline2 = Pipeline([('MinMax', MinMaxScaler()),('t-SNE', TSNEHandler(n_components=2, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])
pipeline3 = Pipeline([('MinMax', MinMaxScaler()),('t-SNE', TSNEHandler(n_components=3, random_state=0)),('Kmeans', KMeans(n_clusters=3, init='random'))])

result1 = pipeline1.fit_predict(iris.data, iris.target)
result2 = pipeline2.fit_predict(iris.data, iris.target)
result3 = pipeline3.fit_predict(iris.data, iris.target)

score1 = normalized_mutual_info_score(iris.target[0], result1)
score2 = normalized_mutual_info_score(iris.target[0], result2)
score3 = normalized_mutual_info_score(iris.target[0], result3)

print('component=1:',score1)
print('component=2:',score2)
print('component=3:',score3)

# 이번에는 PCA와 다르게 component가 2일 때 가장 좋은 결과를 보여줬다.

component=1: 0.736894470189126
component=2: 0.8251627219307877
component=3: 0.22894765296511083
